In [1]:
import tensorflow as tf
import numpy as np
from utils import batch_gather

## test the ion representation model

In [2]:
batch_size = 2
M = 5

In [3]:
#input_location = tf.placeholder(tf.int32, (batch_size, 2, 3))
ion_hidden_represent = tf.placeholder(tf.float32, (batch_size, 2, 3, 4))

In [4]:
#resized_location = tf.reshape(input_location, (-1, 6))
resized_ion_hidden = tf.reshape(ion_hidden_represent, (-1, 6, 4))

In [5]:
output = tf.Variable(tf.zeros((batch_size, 4), dtype=tf.float32),trainable=False, collections=["not_saved"])

In [6]:
unstacked_ion_hidden_represent = tf.unstack(resized_ion_hidden, axis=1)
for i, _ in enumerate(unstacked_ion_hidden_represent):
    #indices = tf.expand_dims(indices, axis=1)
    #gathered_output = batch_gather(params=output, indices=indices)
    output = tf.assign_add(output, unstacked_ion_hidden_represent[i])
    

In [7]:
hr_np = np.arange(2 * 2 * 3 * 4).reshape(2,2,3,4).astype(np.float32)

In [8]:
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(output, feed_dict={ion_hidden_represent: hr_np}))
    print(sess.run(output, feed_dict={ion_hidden_represent: hr_np}))

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: AssignAdd = AssignAdd[T=DT_FLOAT, _class=["loc:@AssignAdd_5"], use_locking=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable, unstack)]]

Caused by op 'AssignAdd', defined at:
  File "/home/rui/.pyenv/versions/3.6.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/rui/.pyenv/versions/3.6.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/rui/.pyenv/versions/3.6.6/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/rui/.pyenv/versions/3.6.6/envs/tensor36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/rui/.pyenv/versions/3.6.6/envs/tensor36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/hom

In [ ]:
ion_hidden_represent